<a href="https://colab.research.google.com/github/felipednegredo/Genetic-TSP-Solver/blob/main/Trabalho_M3_1_I_A_Algoritmo_G%C3%A9netico%E2%80%8B_Caixeiro_Viajante.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Biblotecas

In [ ]:
import matplotlib.pyplot as plt
from google.colab import files
import numpy as np
import random

## Função de retorna as cidades

In [ ]:
# Função para retornar a quantidade de cidades e a matriz de distâncias entre elas
def retorna_cidades(linha):
  contador = 1
  partes = linha.strip().split(';')  # Divide a linha em partes separadas por ';'
  quantidade_cidades = int(partes[0])  # A quantidade de cidades é o primeiro elemento
  distancia_cidades = np.zeros((quantidade_cidades, quantidade_cidades))  # Cria uma matriz de zeros
  np.fill_diagonal(distancia_cidades, 1)  # Preenche a diagonal da matriz com 1s
  # Loop para preencher a matriz de distâncias
  for i in range(quantidade_cidades):
    for j in range(quantidade_cidades):
      if distancia_cidades[i][j] == 0:
        distancia_cidades[i][j] = int(partes[contador])
        distancia_cidades[j][i] = int(partes[contador])
        contador+=1
  np.fill_diagonal(distancia_cidades, 0)  # Preenche a diagonal da matriz com 0s
  return quantidade_cidades, distancia_cidades

## Criação da população

In [ ]:
# Função para criar a população inicial
def criar_populacao(quantidade_cidades,tamanho_populacao):
  populacao = []
  for i in range(tamanho_populacao):
    individuo = np.full(quantidade_cidades, -1)  # Cria um indivíduo com todos os genes iguais a -1
    contador = 0
    while contador < quantidade_cidades:
      posicao = random.randint(0, quantidade_cidades-1)  # Escolhe uma posição aleatória
      if individuo[posicao] == -1:  # Se a posição estiver vazia (-1)
        individuo[posicao] = contador  # Preenche a posição com o contador
        contador+=1
    populacao.append(individuo)  # Adiciona o indivíduo à população
    print(f'Individuo gerado {i} - {str(individuo)}')

  return populacao

## Calculo do Fitness

In [ ]:
# Função para calcular o fitness de cada indivíduo da população
def calcular_fitness(populacao,distancia_cidades,quantidade_cidades):
  # Cria uma lista vazia para armazenar o fitness da população
  populacao_fitness = []
  # Para cada indivíduo na população
  for m in range(len(populacao)):
    # Inicializa o fitness como 0
    fitness = 0
    # Para cada cidade até a penúltima cidade
    for j in range((quantidade_cidades)-1):
      # Adiciona a distância entre a cidade atual e a próxima cidade ao fitness
      fitness += distancia_cidades[populacao[m][j]][populacao[m][j+1]]
    # Adiciona a distância entre a primeira cidade e a penúltima cidade ao fitness
    fitness += distancia_cidades[populacao[m][0]][populacao[m][quantidade_cidades-2]]
    # Adiciona o fitness do indivíduo à lista de fitness da população
    populacao_fitness.append(fitness)
  # Retorna a lista de fitness da população
  return populacao_fitness

## Escolha da população

In [ ]:
# Função para escolher os pais para a reprodução
def escolha_pais(populacao,populacao_fitness):
    inversos = [1/v for v in populacao_fitness]  # Calcula o inverso do fitness
    soma_inversos = sum(inversos)
    valores_normalizados = [round(v / soma_inversos * 100,0) for v in inversos]  # Normaliza os valores para que somem 100
    qtd_agulhas = 4
    agulha = random.randint(1, 100)
    pais = []
    for i in range(qtd_agulhas):
      somador = 0
      for j in range(len(valores_normalizados)):
          somador += valores_normalizados[j]
          if somador >= agulha:
              pais.append(populacao[j])
              break
      agulha+=25
      if agulha>100:
        agulha = agulha - 100

    print("Pais escolhidos para reprodução: " + str(pais))

    return pais

## Reprodução da população

In [ ]:
# Função para reprodução dos indivíduos
def reproducao(populacao,pais,quantidade_cidades):
  # Define a taxa de reprodução
  taxa_reproducao = 95

  # Se um número aleatório entre 1 e 100 for menor ou igual à taxa de reprodução
  if random.randint(1, 100) <= taxa_reproducao:
    # Define a quantidade de filhos
    qtd_filho = 4
    # Cria uma lista vazia para os filhos
    filho = []
    # Para cada filho
    for i in range(qtd_filho):
      # Adiciona um array preenchido com -1 à lista de filhos
      filho.append(np.full(quantidade_cidades, -1))

    if quantidade_cidades%2 == 0:
      delimitador = int(quantidade_cidades/2)-1
    else:
      delimitador = int(quantidade_cidades/2)
    # Para cada cidade até a metade do total de cidades
    for i in range(delimitador):
      # Define os genes dos filhos com base nos genes dos pais
      filho[0][i] = pais[0][i]
      filho[1][i+delimitador] = pais[0][i]
      filho[2][i] = pais[2][i]
      filho[3][i+delimitador] = pais[2][i]

    # Para cada filho
    for m in range(len(filho)):
      # Se o índice do filho for menor que 2, define o pai como 1, senão define como 3
      if m < 2:
        pai= 1
      else:
        pai = 3

      # Para cada cidade
      for i in range(quantidade_cidades):
          # Se o gene do pai não estiver no filho
          if pais[pai][i] not in filho[m]:
              # Para cada cidade
              for j in range(quantidade_cidades):
                  # Se o gene do filho for -1
                  if filho[m][j] == -1:
                      # Define o gene do filho como o gene do pai
                      filho[m][j] = pais[pai][i]
                      break

    # Imprime os filhos e a população
    print('filho ',str(filho))
    print('populacao',str(populacao))
    # Para cada filho
    for i in range(len(filho)):
      # Adiciona o filho à população
      populacao.append(filho[i])

  # Retorna a população
  return populacao

## Mutação da Reprodução

In [ ]:
# Função para realizar a mutação em um indivíduo da população
def mutacao(populacao):
  # Define a taxa de mutação como 5%
  taxa_mutacao = 5
  # Se um número aleatório entre 1 e 100 for menor ou igual à taxa de mutação
  if random.randint(1, 100) <= taxa_mutacao:
    # Seleciona um indivíduo aleatório da população para sofrer mutação
    individuo_mutado = random.randint(0, len(populacao)-1)

    # Seleciona uma posição aleatória dentro do indivíduo para sofrer mutação
    pos_1 = random.randint(0, len(populacao[individuo_mutado])-1)

    # Inicializa a segunda posição como a primeira
    pos_2 = pos_1

    # Enquanto a segunda posição for igual à primeira
    while pos_1 == pos_2:
      # Seleciona uma nova posição aleatória dentro do indivíduo
      pos_2 = random.randint(0, len(populacao[individuo_mutado])-1)

    # Troca os valores nas duas posições selecionadas no indivíduo
    temp = populacao[individuo_mutado][pos_1]
    populacao[individuo_mutado][pos_1] =  populacao[individuo_mutado][pos_2]
    populacao[individuo_mutado][pos_2] = temp

  # Retorna a população após a mutação
  return populacao

## Redução da população

In [ ]:
# Função para simular um evento catastrófico que reduz a população
def meteoro(populacao,populacao_fitness,tamanho_populacao):
  # Imprime o fitness da população
  print(populacao_fitness)
  print()
  # Enquanto o tamanho da população for maior que o tamanho desejado
  while len(populacao) > tamanho_populacao:
    # Inicializa o índice e o valor máximo
    indice = -1
    max_valor = 0
    # Para cada indivíduo na população
    for i in range(len(populacao)-1):
      # Se o fitness do indivíduo for maior que o valor máximo
      if max_valor < populacao_fitness[i]:
          # Atualiza o índice e o valor máximo
          indice = i
          max_valor = populacao_fitness[i]

    # Imprime o valor máximo e o índice
    print(max_valor,indice)
    # Se o índice for maior que -1
    if indice > -1:
      # Remove o indivíduo da população
      populacao.pop(indice)
      # Remove o fitness do indivíduo da lista de fitness da população
      populacao_fitness.pop(indice)
  # Retorna a população
  return populacao

## Execução geral

In [ ]:
tamanho_populacao = 8

# Fazer o upload do arquivo
uploaded = files.upload()
# O nome do arquivo será a chave do dicionário uploaded
nome_arquivo = list(uploaded.keys())[0]
print(f'Arquivo "{nome_arquivo}" foi carregado.')

# Ler o arquivo linha por linha
with open(nome_arquivo, 'r') as file:
    for linha in file:
      quantidade_cidades, distancia_cidades = retorna_cidades(linha)
      print(f'Quantidade de cidades: {quantidade_cidades}')
      print(f'Matriz de distâncias: {distancia_cidades}')
      print()
      print("Geração da população inicial")
      populacao = criar_populacao(quantidade_cidades,tamanho_populacao)
      print()
      print("Fitness da população inicial")
      populacao_fitness = calcular_fitness(populacao,distancia_cidades,quantidade_cidades)
      print()

      print("Nova geração")
      pais = escolha_pais(populacao,populacao_fitness)
      print()
      print("Reprodução")
      populacao = reproducao(populacao,pais,quantidade_cidades)
      print()
      print("Mutação")
      populacao = mutacao(populacao)
      print()
      print("Nova geração")
      populacao_fitness = calcular_fitness(populacao,distancia_cidades,quantidade_cidades)
      print()
      print("Metero")
      print('antes do meteoro -> ',populacao)
      populacao = meteoro(populacao,populacao_fitness,tamanho_populacao)
      print('depois do meteoro -> ',populacao)
      populacao_fitness = calcular_fitness(populacao,distancia_cidades,quantidade_cidades)
      print()
      print("Fitness da população final")
      print(populacao_fitness)
      print()


Saving Novo(a) Text Document.txt to Novo(a) Text Document (15).txt
Arquivo "Novo(a) Text Document (15).txt" foi carregado.
Quantidade de cidades: 5
Matriz de distâncias: [[ 0. 10. 15.  5.  7.]
 [10.  0. 20.  9. 11.]
 [15. 20.  0. 10. 18.]
 [ 5.  9. 10.  0. 19.]
 [ 7. 11. 18. 19.  0.]]

Geração da população inicial
Individuo gerado 0 - [4 3 1 0 2]
Individuo gerado 1 - [0 2 1 3 4]
Individuo gerado 2 - [1 4 3 0 2]
Individuo gerado 3 - [0 2 4 3 1]
Individuo gerado 4 - [0 2 3 4 1]
Individuo gerado 5 - [0 2 3 4 1]
Individuo gerado 6 - [0 2 4 1 3]
Individuo gerado 7 - [4 1 0 2 3]

Fitness da população inicial
populacao [array([4, 3, 1, 0, 2]), array([0, 2, 1, 3, 4]), array([1, 4, 3, 0, 2]), array([0, 2, 4, 3, 1]), array([0, 2, 3, 4, 1]), array([0, 2, 3, 4, 1]), array([0, 2, 4, 1, 3]), array([4, 1, 0, 2, 3])]
populacao fit  [60.0, 68.0, 60.0, 66.0, 62.0, 62.0, 63.0, 64.0]

Nova geração
Pais escolhidos para reprodução: [array([4, 3, 1, 0, 2]), array([1, 4, 3, 0, 2]), array([0, 2, 3, 4, 1]), arr